In [3]:
from gensim.models import Doc2Vec, Word2Vec
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
pd.set_option('display.max_colwidth', -1)

In [4]:
dirpath = "C:/Users/D072202/DeepAnyMatch/DeepAnyMatch/result_data/hilti/HILTI_emb_best_2019_06_19_16_18_15_282440/"
model = Word2Vec.load(dirpath+"w2v.model")
documents_ids_A = dict()
documents_ids_B = dict()
training_corpus = dict()


possible_matches = dict()
with open('C:/Users/D072202/DeepAnyMatch/DeepAnyMatch/data/sap_hilti_data/balanced_walks/possible_matches.csv') as f:
    for line in f:
        line = line.replace("\n","").split("\t")
        if line[0] in possible_matches.keys():
            possible_matches[line[0]].append(line[1])
        else:
            possible_matches[line[0]] = [line[1]]
            
        if line[1] in possible_matches.keys():
            possible_matches[line[1]].append(line[0])
        else:
            possible_matches[line[1]] = [line[0]]

In [8]:
def get_possible_matches(nid):
    out = list()
    for index, key in documents_ids_B.items():
        if key in possible_matches[nid]:
            out.append(index)
    return out


In [9]:
nodeid = 'http://rdata2graph.sap.com/hilti_erp/torx_soc_re_scr_sn65_3x25_black#63ca4fe8-20ca-4115-9b52-05ca0bed27d3'
#nodeid = 'http://rdata2graph.sap.com/hilti_erp/property/t179.description'
#nodeid = 'http://rdata2graph.sap.com/hilti_erp/case_te_40___typ_037#bb007577-546b-4906-9fc9-4830fba5c0a9'

print("Targeted "+ nodeid)

Targeted http://rdata2graph.sap.com/hilti_erp/torx_soc_re_scr_sn65_3x25_black#63ca4fe8-20ca-4115-9b52-05ca0bed27d3


In [7]:
tmp = pd.read_csv(dirpath+"stratified_embeddings.csv", sep=",")
classes = set(tmp.category)
classes1 = [x for x in classes if 'hilti_erp' in x]
classes2 = [x for x in classes if 'hilti_web' in x]


# In[40]:


properties = [x for x in set(tmp.loc[tmp.category == 'http://www.w3.org/2000/01/rdf-schema#property'].label) if 'mara_fert.' not in x]
properties1 = [x for x in properties if 'hilti_erp' in x]
properties2 = [x for x in properties if 'hilti_web' in x]


In [164]:
properties1=[nodeid]

In [183]:
properties2

['http://rdata2graph.sap.com/hilti_web/property/categories.description',
 'http://rdata2graph.sap.com/hilti_web/property/products.name',
 'http://rdata2graph.sap.com/hilti_web/property/categories.id',
 'http://rdata2graph.sap.com/hilti_web/property/products.description',
 'http://rdata2graph.sap.com/hilti_web/property/products.pid',
 'http://rdata2graph.sap.com/hilti_web/property/categories.name']

In [196]:
all_sorted=None
for nodeid in properties1:
    #try:
        #print('Closest in terms of cosine similarity:')
        vecs = model.wv[np.array(get_possible_matches(nodeid))]
        x = cosine_similarity(np.array([model.wv[nodeid]]), vecs)
        x = np.concatenate((x, np.array([get_possible_matches(nodeid)])), axis=0)
        sorted_x = pd.DataFrame(x).T.sort_values(by=[0], ascending=False)
        sorted_x.loc[:,'score'] = 0
        ctr = 1
        for index, row in sorted_x.iterrows():
            print(row[1] + " - " + str(row[0]))
            sorted_x.loc[index, 'score'] = row['score'] + 1/ctr
            ctr += 1
        sorted_x.loc[:,'cosine_similarity'] = sorted_x[0]
        #print('Closest in terms of Euclidean distance:')
        sorted_x2 = sorted_x
        vecs = model.wv[np.array(get_possible_matches(nodeid))]
        x = euclidean_distances(np.array([model.wv[nodeid]]), vecs)
        x = np.concatenate((x, np.array([get_possible_matches(nodeid)])), axis=0)
        sorted_x = pd.DataFrame(x).T.sort_values(by=[0], ascending=True)
        sorted_x.loc[:,'score'] = 0
        ctr = 1
        for index, row in sorted_x.iterrows():
            print(row[1] + " - " + str(row[0]))
            sorted_x.loc[index, 'score'] = row['score'] + 1/ctr
            ctr += 1
        sorted_x.loc[:,'euclid_dist'] = sorted_x[0]
        #print('Closest in sum:')
        x = sorted_x2.merge(sorted_x, left_on=[1], right_on=[1])
        x.loc[:,'total_score'] = x['score_x']#x['score_x'] + x['score_y']
        tmp = x.sort_values(by=['total_score'], ascending=False)
        tmp.loc[:,'nodeid'] = nodeid
        all_sorted = mergedf(all_sorted, tmp)
    #except:
    #    pass

http://rdata2graph.sap.com/hilti_web/property/categories.id - 0.6091314
http://rdata2graph.sap.com/hilti_web/property/categories.name - 0.58044696
http://rdata2graph.sap.com/hilti_web/property/categories.description - 0.57773083
http://rdata2graph.sap.com/hilti_web/property/products.name - 0.5231369
http://rdata2graph.sap.com/hilti_web/property/products.pid - 0.5196886
http://rdata2graph.sap.com/hilti_web/property/products.description - 0.51966494
http://rdata2graph.sap.com/hilti_web/property/categories.id - 3.1806383
http://rdata2graph.sap.com/hilti_web/property/categories.name - 3.4074476
http://rdata2graph.sap.com/hilti_web/property/products.name - 3.4723933
http://rdata2graph.sap.com/hilti_web/property/categories.description - 3.4853716
http://rdata2graph.sap.com/hilti_web/property/products.description - 3.492204
http://rdata2graph.sap.com/hilti_web/property/products.pid - 3.4927726
http://rdata2graph.sap.com/hilti_web/property/products.description - 0.22199357
http://rdata2graph.s

http://rdata2graph.sap.com/hilti_web/property/products.description - 0.22126852
http://rdata2graph.sap.com/hilti_web/property/products.pid - 0.2189769
http://rdata2graph.sap.com/hilti_web/property/products.name - 0.217867
http://rdata2graph.sap.com/hilti_web/property/categories.description - 0.14116389
http://rdata2graph.sap.com/hilti_web/property/categories.name - 0.14080209
http://rdata2graph.sap.com/hilti_web/property/categories.id - 0.1337614
http://rdata2graph.sap.com/hilti_web/property/products.description - 4.750385
http://rdata2graph.sap.com/hilti_web/property/products.name - 4.752427
http://rdata2graph.sap.com/hilti_web/property/products.pid - 4.7578793
http://rdata2graph.sap.com/hilti_web/property/categories.id - 4.984946
http://rdata2graph.sap.com/hilti_web/property/categories.name - 5.08875
http://rdata2graph.sap.com/hilti_web/property/categories.description - 5.1523247
http://rdata2graph.sap.com/hilti_web/property/products.description - 0.2263507
http://rdata2graph.sap.com

http://rdata2graph.sap.com/hilti_web/property/categories.id - 3.20256
http://rdata2graph.sap.com/hilti_web/property/categories.name - 3.4220645
http://rdata2graph.sap.com/hilti_web/property/categories.description - 3.4988296
http://rdata2graph.sap.com/hilti_web/property/products.name - 3.510324
http://rdata2graph.sap.com/hilti_web/property/products.pid - 3.529725
http://rdata2graph.sap.com/hilti_web/property/products.description - 3.5298352
http://rdata2graph.sap.com/hilti_web/property/categories.description - -0.07727175
http://rdata2graph.sap.com/hilti_web/property/categories.name - -0.07431109
http://rdata2graph.sap.com/hilti_web/property/products.description - -0.07196008
http://rdata2graph.sap.com/hilti_web/property/products.pid - -0.07078
http://rdata2graph.sap.com/hilti_web/property/products.name - -0.06894407
http://rdata2graph.sap.com/hilti_web/property/categories.id - -0.0603412
http://rdata2graph.sap.com/hilti_web/property/categories.id - 5.177349
http://rdata2graph.sap.com/

http://rdata2graph.sap.com/hilti_web/property/categories.id - 0.6051039
http://rdata2graph.sap.com/hilti_web/property/categories.name - 0.5728295
http://rdata2graph.sap.com/hilti_web/property/categories.description - 0.5705217
http://rdata2graph.sap.com/hilti_web/property/products.name - 0.52751833
http://rdata2graph.sap.com/hilti_web/property/products.pid - 0.5244155
http://rdata2graph.sap.com/hilti_web/property/products.description - 0.5241459
http://rdata2graph.sap.com/hilti_web/property/categories.id - 3.193702
http://rdata2graph.sap.com/hilti_web/property/categories.name - 3.4318175
http://rdata2graph.sap.com/hilti_web/property/products.name - 3.4579525
http://rdata2graph.sap.com/hilti_web/property/products.pid - 3.4772952
http://rdata2graph.sap.com/hilti_web/property/products.description - 3.4774854
http://rdata2graph.sap.com/hilti_web/property/categories.description - 3.508457
http://rdata2graph.sap.com/hilti_web/property/products.description - 0.16742474
http://rdata2graph.sap.

http://rdata2graph.sap.com/hilti_web/property/categories.id - 0.6041006
http://rdata2graph.sap.com/hilti_web/property/categories.name - 0.5729441
http://rdata2graph.sap.com/hilti_web/property/categories.description - 0.57034206
http://rdata2graph.sap.com/hilti_web/property/products.name - 0.52134484
http://rdata2graph.sap.com/hilti_web/property/products.description - 0.51825505
http://rdata2graph.sap.com/hilti_web/property/products.pid - 0.5178824
http://rdata2graph.sap.com/hilti_web/property/categories.id - 3.1971388
http://rdata2graph.sap.com/hilti_web/property/categories.name - 3.4315398
http://rdata2graph.sap.com/hilti_web/property/products.name - 3.4772096
http://rdata2graph.sap.com/hilti_web/property/products.description - 3.495825
http://rdata2graph.sap.com/hilti_web/property/products.pid - 3.4976132
http://rdata2graph.sap.com/hilti_web/property/categories.description - 3.5091166
http://rdata2graph.sap.com/hilti_web/property/categories.id - 0.6150897
http://rdata2graph.sap.com/h

http://rdata2graph.sap.com/hilti_web/property/products.pid - 3.5213497
http://rdata2graph.sap.com/hilti_web/property/categories.description - 3.565344
http://rdata2graph.sap.com/hilti_web/property/categories.id - 0.578301
http://rdata2graph.sap.com/hilti_web/property/categories.name - 0.55235654
http://rdata2graph.sap.com/hilti_web/property/categories.description - 0.5497111
http://rdata2graph.sap.com/hilti_web/property/products.name - 0.50342935
http://rdata2graph.sap.com/hilti_web/property/products.description - 0.50052226
http://rdata2graph.sap.com/hilti_web/property/products.pid - 0.50030756
http://rdata2graph.sap.com/hilti_web/property/categories.id - 3.283503
http://rdata2graph.sap.com/hilti_web/property/categories.name - 3.499046
http://rdata2graph.sap.com/hilti_web/property/products.name - 3.5344481
http://rdata2graph.sap.com/hilti_web/property/products.description - 3.5523942
http://rdata2graph.sap.com/hilti_web/property/products.pid - 3.553689
http://rdata2graph.sap.com/hilti

In [201]:
tmp2 = all_sorted.sort_values(by=['cosine_similarity'], ascending=[False])
while len(tmp2) > 0:
    row = tmp2.head(1)
    print(row[1].values + " -> " + row.nodeid.values)
    tmp2 = tmp2.loc[~(tmp2[1] == row[1].values[0]) & ~(tmp2['nodeid'] == row['nodeid'].values[0])]

['http://rdata2graph.sap.com/hilti_web/property/categories.id -> http://rdata2graph.sap.com/hilti_erp/property/t023.wwgda']
['http://rdata2graph.sap.com/hilti_web/property/categories.name -> http://rdata2graph.sap.com/hilti_erp/property/t023.abtnr']
['http://rdata2graph.sap.com/hilti_web/property/categories.description -> http://rdata2graph.sap.com/hilti_erp/property/t023.bklas']
['http://rdata2graph.sap.com/hilti_web/property/products.name -> http://rdata2graph.sap.com/hilti_erp/property/t006.primary']
['http://rdata2graph.sap.com/hilti_web/property/products.pid -> http://rdata2graph.sap.com/hilti_erp/property/t023.matkl']
['http://rdata2graph.sap.com/hilti_web/property/products.description -> http://rdata2graph.sap.com/hilti_erp/property/t023.def_schdsc']


In [129]:
def mergedf(df1, df2):
        if df1 is None:
            return df2
        else:
            return df1.append(df2, ignore_index=True)

In [169]:
sorted_x

,0,1,score,euclid_dist
0,12.236134,http://rdata2graph.sap.com/hilti_web/categories,1.0,12.236134
1,9.81989,http://rdata2graph.sap.com/hilti_web/products,0.5,9.81989


In [199]:
all_sorted.sort_values(by=['euclid_dist'], ascending=[True])

,0_x,1,score_x,cosine_similarity,0_y,score_y,euclid_dist,total_score,nodeid
228,0.6242219,http://rdata2graph.sap.com/hilti_web/property/categories.id,1.000000,0.6242219,3.1293268,1.000000,3.1293268,1.000000,http://rdata2graph.sap.com/hilti_erp/property/t023.wwgda
330,0.6215962,http://rdata2graph.sap.com/hilti_web/property/categories.id,1.000000,0.6215962,3.1389866,1.000000,3.1389866,1.000000,http://rdata2graph.sap.com/hilti_erp/property/t023.abtnr
168,0.61908925,http://rdata2graph.sap.com/hilti_web/property/categories.id,1.000000,0.61908925,3.1461315,1.000000,3.1461315,1.000000,http://rdata2graph.sap.com/hilti_erp/property/t179.mandt
78,0.619101,http://rdata2graph.sap.com/hilti_web/property/categories.id,1.000000,0.619101,3.1468308,1.000000,3.1468308,1.000000,http://rdata2graph.sap.com/hilti_erp/property/t023.bklas
54,0.6174795,http://rdata2graph.sap.com/hilti_web/property/categories.id,1.000000,0.6174795,3.1524603,1.000000,3.1524603,1.000000,http://rdata2graph.sap.com/hilti_erp/property/t006.primary
306,0.6153381,http://rdata2graph.sap.com/hilti_web/property/categories.id,1.000000,0.6153381,3.1583529,1.000000,3.1583529,1.000000,http://rdata2graph.sap.com/hilti_erp/property/t006.kzkeh
288,0.6150897,http://rdata2graph.sap.com/hilti_web/property/categories.id,1.000000,0.6150897,3.1608706,1.000000,3.1608706,1.000000,http://rdata2graph.sap.com/hilti_erp/property/t023.price_group
348,0.6105018,http://rdata2graph.sap.com/hilti_web/property/categories.id,1.000000,0.6105018,3.1749732,1.000000,3.1749732,1.000000,http://rdata2graph.sap.com/hilti_erp/property/t006.dimid
0,0.6091314,http://rdata2graph.sap.com/hilti_web/property/categories.id,1.000000,0.6091314,3.1806383,1.000000,3.1806383,1.000000,http://rdata2graph.sap.com/hilti_erp/property/t006.famunit
294,0.6086388,http://rdata2graph.sap.com/hilti_web/property/categories.id,1.000000,0.6086388,3.1812525,1.000000,3.1812525,1.000000,http://rdata2graph.sap.com/hilti_erp/property/v137.mandt


In [48]:
a = np.array([[0.20947344601154327,-0.05213532969355583],[0.20947344601154327,-0.05213532969355583]])
b = np.array([[-0.036627139896154404,-0.24773931503295898],[0.20947344601154327,-0.05213532969355583]])
cosine_similarity(a,b)[0]

array([0.09699644, 1.        ])

In [129]:
nodeid = 'http://rdata2graph.sap.com/hilti_erp/kwik_lock_150_assy#87d1d7cd-5ff6-4adb-9960-e5244a26ea9d'
for index, value in documents_ids_A.items():
    if value == nodeid:
        i = index
nodeid2 = 'http://rdata2graph.sap.com/hilti_web/5_8_kwik_lock_spindle_dd_cs#d059d940-5225-46b5-a5ed-a94535564f64'
for index, value in documents_ids_B.items():
    if value == nodeid2:
        i2 = index
a = np.array([model.infer_vector(training_corpus[i])])#np.array([model.docvecs[i]])
b = np.array([model.infer_vector(training_corpus[i2])])#np.array([model.docvecs[i2]])

In [130]:
print(cosine_similarity(a,b)[0])
print(euclidean_distances(a,b)[0])

[0.78147936]
[2.1596375]


In [117]:
print(cosine_similarity(a,b)[0])
print(euclidean_distances(a,b)[0])

[0.9788133]
[0.8127318]


In [118]:
context = ['http://rdata2graph.sap.com/hilti_erp/stator_110_120v#6dd99497-b3ad-4b7c-9224-519d68aff549', 'sta', 'tat', 'ato', 'tor', '110', '101', '012', '120', '20v', 'http://rdata2graph.sap.com/hilti_erp/#7fc668f7-bc87-4ef0-9686-62c76b4479d2', 'ele', 'lec', 'ect', 'ctr', 'tro', 'ron', 'mod', 'odu', 'dul', '345', 'ele', 'lec', 'ect', 'ctr', 'tro', 'ron', 'mod', 'odu', 'dul', '000', '001', '100', 'z00', '001', 'http://rdata2graph.sap.com/hilti_erp/t023', 'http://rdata2graph.sap.com/hilti_erp/#0e08a169-a93c-4c44-a608-820a1543aa56', 'm', 'm', 'bef', 'efe', 'fes', 'est', 'sti', 'tig', 'igu', 'gun', 'ung', 'ngs', 'gsi', 'sin', 'ind', 'ndu', 'dus', 'ust', 'str', 'tri', 'm', 'http://rdata2graph.sap.com/hilti_erp/v137', 'http://rdata2graph.sap.com/hilti_erp/#08afb57f-34d6-4bf3-9752-df939bff7578', 'eb', 'par', 'art', '9', 'http://rdata2graph.sap.com/hilti_erp/t179', 'http://rdata2graph.sap.com/hilti_erp/mara_fert', 'http://www.w3.org/2000/01/rdf-schema#class', 'http://rdata2graph.sap.com/hilti_erp/stator_110_120v#6dd99497-b3ad-4b7c-9224-519d68aff549', 'sta', 'tat', 'ato', 'tor', '110', '101', '012', '120', '20v', 'http://rdata2graph.sap.com/hilti_erp/#7fc668f7-bc87-4ef0-9686-62c76b4479d2', 'ele', 'lec', 'ect', 'ctr', 'tro', 'ron', 'mod', 'odu', 'dul', '345', 'ele', 'lec', 'ect', 'ctr', 'tro', 'ron', 'mod', 'odu', 'dul', '000', '001', '100', 'z00', '001', 'http://rdata2graph.sap.com/hilti_erp/t023', 'http://rdata2graph.sap.com/hilti_erp/#0e08a169-a93c-4c44-a608-820a1543aa56', 'm', 'm', 'bef', 'efe', 'fes', 'est', 'sti', 'tig', 'igu', 'gun', 'ung', 'ngs', 'gsi', 'sin', 'ind', 'ndu', 'dus', 'ust', 'str', 'tri', 'm', 'http://rdata2graph.sap.com/hilti_erp/v137', 'http://rdata2graph.sap.com/hilti_erp/#08afb57f-34d6-4bf3-9752-df939bff7578', 'eb', 'par', 'art', '9', 'http://rdata2graph.sap.com/hilti_erp/t179', 'http://rdata2graph.sap.com/hilti_erp/mara_fert', 'http://www.w3.org/2000/01/rdf-schema#class']
model.infer_vector(context)

array([0.02674646, 1.2861257 ], dtype=float32)

In [119]:
model.docvecs[823]

array([-0.15971136,  1.0846955 ], dtype=float32)

In [37]:
tmp = pd.read_csv(dirpath+"stratified_embeddings.csv", sep=",")
classes = set(tmp.category)
classes

{'http://rdata2graph.sap.com/hilti_erp/mara_fert',
 'http://rdata2graph.sap.com/hilti_erp/t023',
 'http://rdata2graph.sap.com/hilti_erp/t179',
 'http://rdata2graph.sap.com/hilti_erp/v137',
 'http://rdata2graph.sap.com/hilti_web/categories',
 'http://rdata2graph.sap.com/hilti_web/products',
 'http://www.w3.org/2000/01/rdf-schema#class',
 'http://www.w3.org/2000/01/rdf-schema#property',
 'none'}

In [40]:
properties = [x for x in set(tmp.loc[tmp.category == 'http://www.w3.org/2000/01/rdf-schema#property'].label) if 'mara_fert.' not in x]
properties

['http://rdata2graph.sap.com/hilti_erp/property/t023.abtnr',
 'http://rdata2graph.sap.com/hilti_erp/property/t023.zz_inactive',
 'http://rdata2graph.sap.com/hilti_erp/property/t023.spart',
 'http://rdata2graph.sap.com/hilti_erp/property/t023.id',
 'http://rdata2graph.sap.com/hilti_erp/property/t006.temp_value',
 'http://rdata2graph.sap.com/hilti_erp/property/t023.lref3',
 'http://rdata2graph.sap.com/hilti_erp/property/t006.andec',
 'http://rdata2graph.sap.com/hilti_erp/property/t179.id',
 'http://rdata2graph.sap.com/hilti_erp/property/t023.wwgda',
 'http://rdata2graph.sap.com/hilti_erp/property/t006.press_val',
 'http://rdata2graph.sap.com/hilti_erp/property/v137.id',
 'http://rdata2graph.sap.com/hilti_erp/property/t023.zz_aedat',
 'http://rdata2graph.sap.com/hilti_web/property/categories.name',
 'http://rdata2graph.sap.com/hilti_erp/property/v137.bbref',
 'http://rdata2graph.sap.com/hilti_web/property/categories.description',
 'http://rdata2graph.sap.com/hilti_erp/property/t179.stufe'